In [1]:
from __future__ import print_function
import argparse
import os
import random
import math
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils

c:\Users\david\anaconda3\envs\machine_learning\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [42]:
data_path = os.path.join('C:/Users/david/Documents/Python/data','human_face')
batchsize = 32
workers = 8
nz = 100

# folder dataset
human_dataset = dset.ImageFolder(root=data_path,
                        transform=transforms.Compose([
                            transforms.Resize(64),
                            transforms.CenterCrop(64),
                            transforms.ToTensor(),
                            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                        ]))

assert human_dataset
human_dataloader = torch.utils.data.DataLoader(human_dataset, batch_size=batchsize,
                                        shuffle=True, num_workers=int(workers))

# Grab sample. Depending on dataset, may take a while
sample = next(iter(human_dataloader))[0]

In [43]:
batch_size_measure = sample.size(0)

In [44]:
fixed_noise = torch.randn(batchsize, nz, 1, 1)

In [45]:
def image_to_vector(img, nz):
    preprocess = transforms.Compose([
        transforms.Grayscale(),
        transforms.CenterCrop(int(math.sqrt(nz)))
    ])
    vector = preprocess(img)
    # vector = torch.unsqueeze(vector,1)
    vector = torch.flatten(vector,start_dim=1,end_dim=3)
    return vector[:,:,None,None]

In [46]:
vector = image_to_vector(sample,nz)

In [47]:
class Generator(nn.Module):
    def __init__(self, ngpu,nz,nc,ngf):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.nz = nz
        self.nc = nc
        self.ngf = ngf
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d(     self.nz, self.ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(self.ngf * 8),
            nn.ReLU(True),
            
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(self.ngf * 8, self.ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(self.ngf * 4),
            nn.ReLU(True),
            
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d(self.ngf * 4, self.ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(self.ngf * 2),
            nn.ReLU(True),
            
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d(self.ngf * 2,     self.ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(self.ngf),
            nn.ReLU(True),

            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d( self.ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )

    def forward(self, input):
        if input.is_cuda and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else:
            output = self.main(input)
        return output

In [48]:
netG = Generator(1,nz,3,64)
# netG.apply(weights_init)
netG.load_state_dict(torch.load('C:/Users/david/Documents/Python/DCGAN/output/anime_face/generator_chkpts/netG_epoch_34.pth'))

<All keys matched successfully>

In [51]:
fake = netG(fixed_noise)
vutils.save_image(fake.detach(),
        'noise_output.png',
        normalize=True)
        
fake = netG(vector)
vutils.save_image(vector,
        'vector_input.png',
        normalize=True)
vutils.save_image(fake.detach(),
        'vector_output.png',
        normalize=True)

TypeError: Cannot handle this data type: (1, 1, 100), |u1